
# Set Cover Problem


## Introduction

The set cover problem [[1]](#SetCoverWiki) represents a well-known problem in the fields of combinatorics, computer science, and complexity theory. It is an NP-complete problems.

The problem presents us with a universal set, $\displaystyle U$, and a collection $\displaystyle S$ of subsets of $\displaystyle U$. The goal is to find the smallest possible subfamily, $\displaystyle C \subseteq S$, whose union equals the universal set.

Formally, let's consider a universal set $\displaystyle U = {1, 2, ..., n}$ and a collection $\displaystyle S$ containing $m$ subsets of $\displaystyle U$, $\displaystyle S = {S_1, ..., S_m}$ with $\displaystyle S_i \subseteq U$. The challenge of the set cover problem is to find a subset $\displaystyle C$ of $\displaystyle S$ of minimal size such that $\displaystyle \bigcup_{S_i \in C} S_i = U$.

# Solving with the Classiq platform

We go through the steps of solving the problem with the Classiq platform, using QAOA algorithm [[2](#QAOA)]. The solution is based on defining a pyomo model for the optimization problem we would like to solve.

In [ ]:
from typing import cast

import networkx as nx
import numpy as np
import pyomo.core as pyo
from IPython.display import Markdown, display
from matplotlib import pyplot as plt

## Building the Pyomo model from a graph input

We proceed by defining the pyomo model that will be used on the Classiq platform, using the mathematical formulation defined above:

In [ ]:
import itertools
from typing import List

import pyomo.core as pyo


def set_cover(sub_sets: List[List[int]]) -> pyo.ConcreteModel:
    entire_set = set(itertools.chain(*sub_sets))
    n = max(entire_set)
    num_sets = len(sub_sets)
    assert entire_set == set(
        range(1, n + 1)
    ), f"the union of the subsets is {entire_set} not equal to range(1, {n + 1})"

    model = pyo.ConcreteModel()
    model.x = pyo.Var(range(num_sets), domain=pyo.Binary)

    @model.Constraint(entire_set)
    def independent_rule(model, num):
        return sum(model.x[idx] for idx in range(num_sets) if num in sub_sets[idx]) >= 1

    model.cost = pyo.Objective(expr=sum(model.x.values()), sense=pyo.minimize)

    return model

The model contains:

- Binary variable for each subset (model.x) indicating if it is included in the sub-collection.
- Objective rule – the size of the sub-collection.
- Constraint – the sub-collection covers the original set.

In [ ]:
sub_sets = sub_sets = [
    [1, 2, 3, 4],
    [2, 3, 4, 5],
    [6, 7],
    [8, 9, 10],
    [1, 6, 8],
    [3, 7, 9],
    [4, 7, 10],
    [2, 5, 8],
]

set_cover_model = set_cover(sub_sets)

In [ ]:
set_cover_model.pprint()

## Setting Up the Classiq Problem Instance

In order to solve the Pyomo model defined above, we use the Classiq combinatorial optimization engine. For the quantum part of the QAOA algorithm (`QAOAConfig`) - define the number of repetitions (`num_layers`):

In [ ]:
from classiq import construct_combinatorial_optimization_model
from classiq.applications.combinatorial_optimization import OptimizerConfig, QAOAConfig

qaoa_config = QAOAConfig(num_layers=3, penalty_energy=10)

For the classical optimization part of the QAOA algorithm we define the maximum number of classical iterations (`max_iteration`) and the $\alpha$-parameter (`alpha_cvar`) for running CVaR-QAOA, an improved variation of the QAOA algorithm [[3](#cvar)]:

In [ ]:
optimizer_config = OptimizerConfig(max_iteration=60, alpha_cvar=0.7)

Lastly, we load the model, based on the problem and algorithm parameters, which we can use to solve the problem:

In [ ]:
qmod = construct_combinatorial_optimization_model(
    pyo_model=set_cover_model,
    qaoa_config=qaoa_config,
    optimizer_config=optimizer_config,
)

We also set the quantum backend we want to execute on:

In [ ]:
from classiq import set_execution_preferences
from classiq.execution import ClassiqBackendPreferences, ExecutionPreferences

backend_preferences = ExecutionPreferences(
    backend_preferences=ClassiqBackendPreferences(backend_name="aer_simulator")
)

qmod = set_execution_preferences(qmod, backend_preferences)

In [ ]:
with open("set_cover.qmod", "w") as f:
    f.write(qmod)

## Synthesizing the QAOA Circuit and Solving the Problem

We can now synthesize and view the QAOA circuit (ansatz) used to solve the optimization problem:

In [ ]:
from classiq import show, synthesize

qprog = synthesize(qmod)
show(qprog)

We now solve the problem using the generated circuit by using the `execute` method:

In [ ]:
from classiq import execute

res = execute(qprog).result()

We can check the convergence of the run:

In [ ]:
from classiq.execution import VQESolverResult

vqe_result = res[1].value
vqe_result.convergence_graph

# Optimization Results

We can also examine the statistics of the algorithm:

In [ ]:
import pandas as pd

optimization_result = pd.DataFrame.from_records(res[0].value)
optimization_result.sort_values(by="cost", ascending=True).head(5)

And the histogram:

In [ ]:
optimization_result.hist("cost", weights=optimization_result["probability"])

Let us plot the solution:

In [ ]:
best_solution = optimization_result.solution[optimization_result.cost.idxmin()]

In [ ]:
print(
    f"Quantum Solution: num_sets={int(sum(best_solution))}, sets={[sub_sets[i] for i in range(len(best_solution)) if best_solution[i]]}"
)

Lastly, we can compare to the classical solution of the problem:

In [ ]:
from pyomo.opt import SolverFactory

solver = SolverFactory("couenne")
solver.solve(set_cover_model)

set_cover_model.display()

In [ ]:
classical_solution = [
    pyo.value(set_cover_model.x[i]) for i in range(len(set_cover_model.x))
]

In [ ]:
print(
    f"Classical Solution: num_sets={int(sum(classical_solution))}, sets={[sub_sets[i] for i in range(len(classical_solution)) if classical_solution[i]]}"
)


## References

<a id='SetCoverWiki'>[1]</a>: [Set Cover Problem (Wikipedia)](https://en.wikipedia.org/wiki/Set_cover_problem)

<a id='QAOA'>[2]</a>: [Farhi, Edward, Jeffrey Goldstone, and Sam Gutmann. "A quantum approximate optimization algorithm." arXiv preprint arXiv:1411.4028 (2014).](https://arxiv.org/abs/1411.4028)

<a id='cvar'>[3]</a>: [Barkoutsos, Panagiotis Kl, et al. "Improving variational quantum optimization using CVaR." Quantum 4 (2020): 256.](https://arxiv.org/abs/1907.04769)
